In [52]:
import pandas as pd
import sqlite3

#### Подключение к базе данных

In [53]:
try:
    conn = sqlite3.connect('../data/checking-logs.sqlite')
    print("Успешное подключение к базе данных.")
except sqlite3.Error as e:
    print(f"Ошибка при подключении к базе данных: {e}")
    exit()

Успешное подключение к базе данных.


#### A/B-тестирование

In [54]:
query = """
    SELECT 
        time,
       avg(diff) AS avg_diff
    FROM 
        (SELECT 
            uid,
            CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(test.first_commit_ts)) * 24 as Integer
                ) as diff,
                CASE WHEN test.first_commit_ts < test.first_view_ts
                THEN 'before' ELSE 'after' END AS time
        FROM test LEFT JOIN deadlines on test.labname = deadlines.labs
        WHERE 
            labname !='project1'
        )
    WHERE 
        uid in (SELECT 
                uid
                FROM (SELECT 
                    uid,
                        CASE WHEN test.first_commit_ts < test.first_view_ts
                        THEN 'before' ELSE 'after' END as time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE 
                        labname != 'project1'
                    )
                GROUP BY uid
                HAVING COUNT(DISTINCT time)=2
                )
    GROUP BY time
"""

test_results = pd.read_sql_query(query, conn)
test_results

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [ ]:
query = """
    SELECT 
        time,
       avg(diff) AS avg_diff
    FROM 
        (SELECT 
            uid,
            CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(control.first_commit_ts)) * 24 as Integer
                ) as diff,
                CASE WHEN control.first_commit_ts < control.first_view_ts
                THEN 'before' ELSE 'after' END AS time
        FROM control LEFT JOIN deadlines on control.labname = deadlines.labs
        WHERE 
            labname !='project1'
        )
    WHERE 
        uid in (SELECT 
                uid
                FROM (SELECT 
                    uid,
                        CASE WHEN control.first_commit_ts < control.first_view_ts
                        THEN 'before' ELSE 'after' END as time
                    FROM control
                    LEFT JOIN deadlines ON control.labname=deadlines.labs
                    WHERE 
                        labname != 'project1'
                    )
                GROUP BY uid
                HAVING COUNT(DISTINCT time)=2
                )
    GROUP BY time
"""
control_results = pd.read_sql_query(query, conn)
control_results

,time,avg_diff
0,after,117.636364
1,before,99.464286


#### Гипотеза оказалась верной и страница действительно влияет на поведение студентов?

Предварительные выводы:

* В тестовой группе наблюдается значительное увеличение дельты после первого просмотра новостной ленты (104.6 - 60.56 = 44.04 часа). Это говорит о том, что студенты в тестовой группе стали начинать работать над заданиями раньше после того, как у них появилась возможность просматривать новостную ленту.
* В контрольной группе также наблюдается увеличение дельты, но менее выраженное (117.64 - 99.46 = 18.18 часа). Это может быть связано с другими факторами, не связанными с новостной лентой (например, студенты в целом стали лучше планировать свое время).

    Исходя из представленных результатов можем сделсть предварительный вывод, что гипотеза **подтвердилась.**
    
    _Для подтверждения гипотезы о влиянии новостной ленты, необходимо провести статистический анализ, чтобы убедиться, что наблюдаемые различия являются статистически значимыми и не обусловлены случайностью._

#### Закрытие соединения

In [56]:
conn.close()